In [1]:
import pandas as pd
from itertools import combinations
from geopy.distance import geodesic

# Load the uploaded CSV file to inspect its structure and contents
file_path = 'circuits_full.csv'
circuits_df = pd.read_csv(file_path)

# Display the first few rows to understand the data structure
circuits_df.head()

,circuits,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,circuitId,url,circuitName,locality,country,latitude,longitude
1,albert_park,http://en.wikipedia.org/wiki/Melbourne_Grand_P...,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.8497,144.968
2,americas,http://en.wikipedia.org/wiki/Circuit_of_the_Am...,Circuit of the Americas,Austin,USA,30.1328,-97.6411
3,bahrain,http://en.wikipedia.org/wiki/Bahrain_Internati...,Bahrain International Circuit,Sakhir,Bahrain,26.0325,50.5106
4,baku,http://en.wikipedia.org/wiki/Baku_City_Circuit,Baku City Circuit,Baku,Azerbaijan,40.3725,49.8533


In [2]:
# Reload the data to include the city information
circuits_df_full = pd.read_csv(file_path)

# Set the first row as headers and clean up
circuits_df_full.columns = circuits_df_full.iloc[0]
circuits_df_full = circuits_df_full[1:]

# Select relevant columns: 'circuitName', 'locality', 'latitude', 'longitude'
circuits_df_full = circuits_df_full[['circuitName', 'locality', 'latitude', 'longitude']]
circuits_df_full = circuits_df_full.rename(columns={
    'circuitName': 'Circuit',
    'locality': 'City',
    'latitude': 'Latitude',
    'longitude': 'Longitude'
})

# Ensure Latitude and Longitude are numeric
circuits_df_full['Latitude'] = pd.to_numeric(circuits_df_full['Latitude'], errors='coerce')
circuits_df_full['Longitude'] = pd.to_numeric(circuits_df_full['Longitude'], errors='coerce')

# Remove rows with missing data
circuits_df_full = circuits_df_full.dropna(subset=['Latitude', 'Longitude'])

# Create all pairwise combinations including city information
circuit_pairs_with_city = list(combinations(circuits_df_full.to_dict(orient='records'), 2))

# Calculate distances and include city information
distance_data_with_city = []
for pair in circuit_pairs_with_city:
    circuit1, circuit2 = pair
    coord1 = (circuit1['Latitude'], circuit1['Longitude'])
    coord2 = (circuit2['Latitude'], circuit2['Longitude'])
    distance_km = geodesic(coord1, coord2).kilometers
    distance_data_with_city.append({
        'Circuit 1': circuit1['Circuit'],
        'City 1': circuit1['City'],
        'Circuit 2': circuit2['Circuit'],
        'City 2': circuit2['City'],
        'Distance (km)': distance_km
    })

# Convert to DataFrame for output
circuits_df = pd.DataFrame(distance_data_with_city)

circuits_df

,Circuit 1,City 1,Circuit 2,City 2,Distance (km)
0,Albert Park Grand Prix Circuit,Melbourne,Circuit of the Americas,Austin,14283.314089
1,Albert Park Grand Prix Circuit,Melbourne,Bahrain International Circuit,Sakhir,12105.288729
2,Albert Park Grand Prix Circuit,Melbourne,Baku City Circuit,Baku,12976.043404
3,Albert Park Grand Prix Circuit,Melbourne,Hungaroring,Budapest,15539.467901
4,Albert Park Grand Prix Circuit,Melbourne,Autodromo Enzo e Dino Ferrari,Imola,16082.647352
...,...,...,...,...,...
271,Circuit Gilles Villeneuve,Montreal,Circuit Park Zandvoort,Zandvoort,5486.411164
272,Circuit Gilles Villeneuve,Montreal,Suzuka Circuit,Suzuka,10608.263150
273,Yas Marina Circuit,Abu Dhabi,Circuit Park Zandvoort,Zandvoort,5207.483709
274,Yas Marina Circuit,Abu Dhabi,Suzuka Circuit,Suzuka,7801.971087


In [3]:
# Update the function to revise the distance threshold for Car/Train to 500 km
def estimate_transport_method_revised(distance_km):
    if distance_km < 500:
        return "Car/Train"
    elif distance_km < 1500:
        return "Train/Short Flight"
    else:
        return "Flight"

# Apply the revised function to estimate the transportation method
circuits_df['Estimated Transport Method'] = circuits_df['Distance (km)'].apply(estimate_transport_method_revised)

circuits_df

,Circuit 1,City 1,Circuit 2,City 2,Distance (km),Estimated Transport Method
0,Albert Park Grand Prix Circuit,Melbourne,Circuit of the Americas,Austin,14283.314089,Flight
1,Albert Park Grand Prix Circuit,Melbourne,Bahrain International Circuit,Sakhir,12105.288729,Flight
2,Albert Park Grand Prix Circuit,Melbourne,Baku City Circuit,Baku,12976.043404,Flight
3,Albert Park Grand Prix Circuit,Melbourne,Hungaroring,Budapest,15539.467901,Flight
4,Albert Park Grand Prix Circuit,Melbourne,Autodromo Enzo e Dino Ferrari,Imola,16082.647352,Flight
...,...,...,...,...,...,...
271,Circuit Gilles Villeneuve,Montreal,Circuit Park Zandvoort,Zandvoort,5486.411164,Flight
272,Circuit Gilles Villeneuve,Montreal,Suzuka Circuit,Suzuka,10608.263150,Flight
273,Yas Marina Circuit,Abu Dhabi,Circuit Park Zandvoort,Zandvoort,5207.483709,Flight
274,Yas Marina Circuit,Abu Dhabi,Suzuka Circuit,Suzuka,7801.971087,Flight


In [4]:
import numpy as np


# Define a function to estimate cost based on transportation method and distance
def estimate_cost(distance_km, transport_method):
    # Define base cost per km and variation based on transport method
    if transport_method == "Car/Train":
        cost_per_km = 2  # Base cost in $ per km
        variation = 0.02
    elif transport_method == "Train/Short Flight":
        cost_per_km = 6
        variation = 0.05
    else:  # Flight
        cost_per_km = 10
        variation = 0.1
    
    # Generate a random cost with normal distribution around the mean
    mean_cost = distance_km * cost_per_km
    random_cost = np.random.normal(mean_cost, mean_cost * variation)
    return max(random_cost, 0)  # Ensure cost is not negative

# Apply the cost estimation to each row
circuits_df['Estimated Cost ($)'] = circuits_df.apply(
    lambda row: estimate_cost(row['Distance (km)'], row['Estimated Transport Method']),
    axis=1
)

circuits_df

,Circuit 1,City 1,Circuit 2,City 2,Distance (km),Estimated Transport Method,Estimated Cost ($)
0,Albert Park Grand Prix Circuit,Melbourne,Circuit of the Americas,Austin,14283.314089,Flight,159065.867438
1,Albert Park Grand Prix Circuit,Melbourne,Bahrain International Circuit,Sakhir,12105.288729,Flight,123126.193290
2,Albert Park Grand Prix Circuit,Melbourne,Baku City Circuit,Baku,12976.043404,Flight,102439.361869
3,Albert Park Grand Prix Circuit,Melbourne,Hungaroring,Budapest,15539.467901,Flight,164696.197800
4,Albert Park Grand Prix Circuit,Melbourne,Autodromo Enzo e Dino Ferrari,Imola,16082.647352,Flight,156449.162358
...,...,...,...,...,...,...,...
271,Circuit Gilles Villeneuve,Montreal,Circuit Park Zandvoort,Zandvoort,5486.411164,Flight,52380.380528
272,Circuit Gilles Villeneuve,Montreal,Suzuka Circuit,Suzuka,10608.263150,Flight,111835.448839
273,Yas Marina Circuit,Abu Dhabi,Circuit Park Zandvoort,Zandvoort,5207.483709,Flight,55813.995122
274,Yas Marina Circuit,Abu Dhabi,Suzuka Circuit,Suzuka,7801.971087,Flight,86867.998534


In [5]:
# Create reverse pairs for each entry
reverse_pairs = circuits_df.copy()
reverse_pairs.rename(columns={
    'Circuit 1': 'Circuit 2',
    'City 1': 'City 2',
    'Circuit 2': 'Circuit 1',
    'City 2': 'City 1'
}, inplace=True)

# Combine the original and reverse pairs
full_pairs_df = pd.concat([circuits_df, reverse_pairs], ignore_index=True)
circuits_df = full_pairs_df

circuits_df

,Circuit 1,City 1,Circuit 2,City 2,Distance (km),Estimated Transport Method,Estimated Cost ($)
0,Albert Park Grand Prix Circuit,Melbourne,Circuit of the Americas,Austin,14283.314089,Flight,159065.867438
1,Albert Park Grand Prix Circuit,Melbourne,Bahrain International Circuit,Sakhir,12105.288729,Flight,123126.193290
2,Albert Park Grand Prix Circuit,Melbourne,Baku City Circuit,Baku,12976.043404,Flight,102439.361869
3,Albert Park Grand Prix Circuit,Melbourne,Hungaroring,Budapest,15539.467901,Flight,164696.197800
4,Albert Park Grand Prix Circuit,Melbourne,Autodromo Enzo e Dino Ferrari,Imola,16082.647352,Flight,156449.162358
...,...,...,...,...,...,...,...
547,Circuit Park Zandvoort,Zandvoort,Circuit Gilles Villeneuve,Montreal,5486.411164,Flight,52380.380528
548,Suzuka Circuit,Suzuka,Circuit Gilles Villeneuve,Montreal,10608.263150,Flight,111835.448839
549,Circuit Park Zandvoort,Zandvoort,Yas Marina Circuit,Abu Dhabi,5207.483709,Flight,55813.995122
550,Suzuka Circuit,Suzuka,Yas Marina Circuit,Abu Dhabi,7801.971087,Flight,86867.998534


In [6]:
# Add Spa to the mapping and update the continent information
def map_to_continent_final(city):
    city_to_continent = {
        "Melbourne": "Oceania",
        "Austin": "North America",
        "Sakhir": "Asia",
        "Baku": "Asia",
        "Budapest": "Europe",
        "Imola": "Europe",
        "Silverstone": "Europe",
        "London": "Europe",
        "Donington": "Europe",
        "São Paulo": "South America",
        "Jeddah": "Asia",
        "Al Daayen": "Asia",
        "Marina Bay": "Asia",
        "Miami": "North America",
        "Monte-Carlo": "Europe",
        "Monza": "Europe",
        "Barcelona": "Europe",
        "Spielberg": "Europe",
        "Mexico City": "North America",
        "Shanghai": "Asia",
        "Las Vegas": "North America",
        "Abu Dhabi": "Asia",
        "Zandvoort": "Europe",
        "Spa": "Europe",
        "Montreal": "North America",
        "Suzuka": "Asia"
    }
    return city_to_continent.get(city, "Unknown")

# Apply the updated continent mapping to both Circuit 1 and Circuit 2 cities
circuits_df['Continent 1'] = circuits_df['City 1'].apply(map_to_continent_final)
circuits_df['Continent 2'] = circuits_df['City 2'].apply(map_to_continent_final)

circuits_df

,Circuit 1,City 1,Circuit 2,City 2,Distance (km),Estimated Transport Method,Estimated Cost ($),Continent 1,Continent 2
0,Albert Park Grand Prix Circuit,Melbourne,Circuit of the Americas,Austin,14283.314089,Flight,159065.867438,Oceania,North America
1,Albert Park Grand Prix Circuit,Melbourne,Bahrain International Circuit,Sakhir,12105.288729,Flight,123126.193290,Oceania,Asia
2,Albert Park Grand Prix Circuit,Melbourne,Baku City Circuit,Baku,12976.043404,Flight,102439.361869,Oceania,Asia
3,Albert Park Grand Prix Circuit,Melbourne,Hungaroring,Budapest,15539.467901,Flight,164696.197800,Oceania,Europe
4,Albert Park Grand Prix Circuit,Melbourne,Autodromo Enzo e Dino Ferrari,Imola,16082.647352,Flight,156449.162358,Oceania,Europe
...,...,...,...,...,...,...,...,...,...
547,Circuit Park Zandvoort,Zandvoort,Circuit Gilles Villeneuve,Montreal,5486.411164,Flight,52380.380528,Europe,North America
548,Suzuka Circuit,Suzuka,Circuit Gilles Villeneuve,Montreal,10608.263150,Flight,111835.448839,Asia,North America
549,Circuit Park Zandvoort,Zandvoort,Yas Marina Circuit,Abu Dhabi,5207.483709,Flight,55813.995122,Europe,Asia
550,Suzuka Circuit,Suzuka,Yas Marina Circuit,Abu Dhabi,7801.971087,Flight,86867.998534,Asia,Asia


In [7]:
# Define a mapping for city to country
def map_to_country(city):
    city_to_country = {
        "Melbourne": "Australia",
        "Austin": "USA",
        "Sakhir": "Bahrain",
        "Baku": "Azerbaijan",
        "Budapest": "Hungary",
        "Imola": "Italy",
        "Silverstone": "United Kingdom",
        "London": "United Kingdom",
        "Donington": "United Kingdom",
        "São Paulo": "Brazil",
        "Jeddah": "Saudi Arabia",
        "Al Daayen": "Qatar",
        "Marina Bay": "Singapore",
        "Miami": "USA",
        "Monte-Carlo": "Monaco",
        "Monza": "Italy",
        "Barcelona": "Spain",
        "Spielberg": "Austria",
        "Mexico City": "Mexico",
        "Shanghai": "China",
        "Las Vegas": "USA",
        "Abu Dhabi": "UAE",
        "Zandvoort": "Netherlands",
        "Spa": "Belgium",
        "Montreal": "Canada",
        "Suzuka": "Japan"
    }
    return city_to_country.get(city, "Unknown")

# Apply the country mapping to both Circuit 1 and Circuit 2 cities
circuits_df['Country 1'] = circuits_df['City 1'].apply(map_to_country)
circuits_df['Country 2'] = circuits_df['City 2'].apply(map_to_country)

circuits_df

,Circuit 1,City 1,Circuit 2,City 2,Distance (km),Estimated Transport Method,Estimated Cost ($),Continent 1,Continent 2,Country 1,Country 2
0,Albert Park Grand Prix Circuit,Melbourne,Circuit of the Americas,Austin,14283.314089,Flight,159065.867438,Oceania,North America,Australia,USA
1,Albert Park Grand Prix Circuit,Melbourne,Bahrain International Circuit,Sakhir,12105.288729,Flight,123126.193290,Oceania,Asia,Australia,Bahrain
2,Albert Park Grand Prix Circuit,Melbourne,Baku City Circuit,Baku,12976.043404,Flight,102439.361869,Oceania,Asia,Australia,Azerbaijan
3,Albert Park Grand Prix Circuit,Melbourne,Hungaroring,Budapest,15539.467901,Flight,164696.197800,Oceania,Europe,Australia,Hungary
4,Albert Park Grand Prix Circuit,Melbourne,Autodromo Enzo e Dino Ferrari,Imola,16082.647352,Flight,156449.162358,Oceania,Europe,Australia,Italy
...,...,...,...,...,...,...,...,...,...,...,...
547,Circuit Park Zandvoort,Zandvoort,Circuit Gilles Villeneuve,Montreal,5486.411164,Flight,52380.380528,Europe,North America,Netherlands,Canada
548,Suzuka Circuit,Suzuka,Circuit Gilles Villeneuve,Montreal,10608.263150,Flight,111835.448839,Asia,North America,Japan,Canada
549,Circuit Park Zandvoort,Zandvoort,Yas Marina Circuit,Abu Dhabi,5207.483709,Flight,55813.995122,Europe,Asia,Netherlands,UAE
550,Suzuka Circuit,Suzuka,Yas Marina Circuit,Abu Dhabi,7801.971087,Flight,86867.998534,Asia,Asia,Japan,UAE


In [24]:
import pandas as pd
from gurobipy import Model, GRB, quicksum

# Load and clean data
def load_and_clean_data(df):
    data = df
    data_cleaned = data[["Circuit 1", "City 1", "Circuit 2", "City 2", "Estimated Cost ($)", "Continent 1", "Continent 2", "Country 1", "Country 2"]].rename(
        columns={"Circuit 1": "From", "City 1": "From City", "Circuit 2": "To", "City 2": "To City", "Estimated Cost ($)": "Cost", "Continent 1": "From Continent", "Continent 2": "To Continent", "Country 1": "From Country", "Country 2": "To Country"}
    )
    data_cleaned["Cost"] = pd.to_numeric(data_cleaned["Cost"], errors="coerce")
    data_cleaned["Cost"] = data_cleaned["Cost"].round(2)  # Round cost to 2 decimal places
    data_cleaned = data_cleaned.dropna()
    data_cleaned = data_cleaned.drop_duplicates(subset=["From", "To"])
    return data_cleaned

# Create the optimization model
def solve_tsp(data_cleaned):
    # Extract the unique circuits for node indexing
    circuits = pd.concat([data_cleaned["From"], data_cleaned["To"]]).unique()
    num_circuits = len(circuits)

    # Create a mapping for circuits to indices and vice versa
    circuit_to_idx = {circuit: idx for idx, circuit in enumerate(circuits)}
    idx_to_circuit = {idx: circuit for circuit, idx in circuit_to_idx.items()}

    # Create the cost matrix
    cost_matrix = pd.DataFrame(0, index=range(num_circuits), columns=range(num_circuits))
    for _, row in data_cleaned.iterrows():
        i, j = circuit_to_idx[row["From"]], circuit_to_idx[row["To"]]
        cost_matrix.at[i, j] = row["Cost"]

    # Initialize the Gurobi model
    model = Model("F1_Track_TSP")

    # Add decision variables: x[i, j] is 1 if route from i to j is selected, 0 otherwise
    x = model.addVars(num_circuits, num_circuits, vtype=GRB.BINARY, name="x")

    # Add auxiliary variables for subtour elimination (MTZ constraints)
    u = model.addVars(num_circuits, vtype=GRB.CONTINUOUS, lb=0, ub=num_circuits-1, name="u")

    # Objective function: Minimize total cost
    model.setObjective(quicksum(cost_matrix.at[i, j] * x[i, j] for i in range(num_circuits) for j in range(num_circuits)), GRB.MINIMIZE)

    # Constraints
    for i in range(num_circuits):
        if i != circuit_to_idx["Albert Park Grand Prix Circuit"] and i != circuit_to_idx["Yas Marina Circuit"]:
            # Ensure exactly one incoming and one outgoing connection for all nodes except Bahrain and Abu Dhabi
            model.addConstr(quicksum(x[i, j] for j in range(num_circuits) if j != i) == 1, name=f"out_{i}")
            model.addConstr(quicksum(x[j, i] for j in range(num_circuits) if j != i) == 1, name=f"in_{i}")

    # Start and end points
    bahrain_idx = circuit_to_idx["Albert Park Grand Prix Circuit"]
    abu_dhabi_idx = circuit_to_idx["Yas Marina Circuit"]
    model.addConstr(quicksum(x[bahrain_idx, j] for j in range(num_circuits) if j != bahrain_idx) == 1, name="start_bahrain")
    model.addConstr(quicksum(x[j, bahrain_idx] for j in range(num_circuits) if j != bahrain_idx) == 0, name="no_flow_into_bahrain")
    model.addConstr(quicksum(x[j, abu_dhabi_idx] for j in range(num_circuits) if j != abu_dhabi_idx) == 1, name="last_abu_dhabi")
    model.addConstr(quicksum(x[abu_dhabi_idx, j] for j in range(num_circuits) if j != abu_dhabi_idx) == 0, name="no_flow_out_of_abu_dhabi")

    # Subtour elimination constraints (MTZ formulation)
    for i in range(1, num_circuits):
        for j in range(1, num_circuits):
            if i != j :
                model.addConstr(u[i] - u[j] + num_circuits * x[i, j] <= num_circuits - 1, name=f"subtour_{i}_{j}")
    model.addConstr(u[bahrain_idx] == 0, "fix_start_node")    
    
    # Add schedule-specific constraints
            
    # 1. No two consecutive circuits in the same country
    countries = data_cleaned.groupby("From Country")["From"].unique()
    for country, circuits in countries.items():
        if len(circuits) > 1:
            indices = [circuit_to_idx[circuit] for circuit in circuits]
            for i in range(len(indices)):
                for j in range(len(indices)):
                    if i != j:
                        model.addConstr(x[indices[i], indices[j]] == 0, name=f"no_consecutive_{country}_{i}_{j}")

    # 2. Two Italy circuits(Imola & Monza) have to be seperated into two half of season
    Imola_idx = circuit_to_idx["Autodromo Enzo e Dino Ferrari"]
    Monza_idx = circuit_to_idx["Autodromo Nazionale di Monza"]
    model.addConstr(u[Imola_idx] <= 5, "Imola_First_Half")
    model.addConstr(u[Monza_idx] >= 13, "Monza_Second_Half")
    
    # Optimize the model
    model.optimize()
    

    # Extract the optimal tour
    if model.status == GRB.OPTIMAL:
        # Print the selected x[i, j] edges
        print("\nSelected Edges (x[i, j]):")
        for i in range(num_circuits):
            for j in range(num_circuits):
                if x[i, j].x > 0.5:  # Print only edges with x[i, j] = 1
                    print(f"Edge: {idx_to_circuit[i]} -> {idx_to_circuit[j]} (x[{i}, {j}] = {x[i, j].x})")

        # Extract and print the tour
        tour = []
        current = bahrain_idx
        visited = set()

        while current != abu_dhabi_idx:
            tour.append(current)
            visited.add(current)

            # Find the next city
            next_cities = [j for j in range(num_circuits) if x[current, j].x > 0.5 and j not in visited]
            if not next_cities:
                print(f"No valid outgoing edge from {idx_to_circuit[current]}. Check constraints or data.")
                break

            next_city = next_cities[0]
            current = next_city

        # Add Abu Dhabi as the final stop
        if current == abu_dhabi_idx:
            tour.append(current)

        # Map indices back to circuit names and cities
        optimal_tour = [(idx_to_circuit[idx], data_cleaned.loc[data_cleaned["From"] == idx_to_circuit[idx], "From City"].iloc[0]) for idx in tour]
        optimal_cost = model.objVal

        print("\nOptimal Tour:")
        for circuit, city in optimal_tour:
            print(f"{circuit} ({city})")
        print("\nTotal Cost:", round(optimal_cost, 2))
        
        return None,None

    else:
        print("Model is infeasible or no solution found.")

# Main execution
if __name__ == "__main__":
    data_cleaned = load_and_clean_data(circuits_df)
    optimal_tour, optimal_cost = solve_tsp(data_cleaned)

    if optimal_tour:
        print("Optimal Tour:")
        for circuit, city in optimal_tour:
            print(f"{circuit} ({city})")
        print("Total Cost:", round(optimal_cost, 2))

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[arm] - Darwin 24.0.0 24A335)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 565 rows, 600 columns and 2633 nonzeros
Model fingerprint: 0x881cdcf3
Variable types: 24 continuous, 576 integer (576 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [2e+02, 2e+05]
  Bounds range     [1e+00, 2e+01]
  RHS range        [1e+00, 2e+01]
Presolve removed 44 rows and 79 columns
Presolve time: 0.02s
Presolved: 521 rows, 521 columns, 2402 nonzeros
Variable types: 22 continuous, 499 integer (499 binary)

Root relaxation: objective 4.045238e+05, 81 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 404523.848    0   30          - 404523.848      -     -    0s
H    0     0                 